In [ ]:
# Opti 2 Assmgt 1

In [ ]:
import time
from tqdm.notebook import tqdm

In [ ]:
from google.colab import drive
import os

# 1. Mount Google Drive
drive.mount('/content/drive')

# 2. Define a folder path in your Drive where you want the files
#    (Change 'Connect4_Project' to whatever folder name you prefer)
save_path = '/content/drive/My Drive/Connect4_Project'

# 3. Create the folder if it doesn't exist
if not os.path.exists(save_path):
    os.makedirs(save_path)
    print(f"Created new folder: {save_path}")
else:
    print(f"Saving to existing folder: {save_path}")

Mounted at /content/drive
Saving to existing folder: /content/drive/My Drive/Connect4_Project


In [ ]:
import numpy as np
import random

def update_board(board_temp, color, column):
    board = board_temp.copy()
    # Calculates the row to drop the piece based on the column sum
    colsum = abs(board[0,column])+abs(board[1,column])+abs(board[2,column])+abs(board[3,column])+abs(board[4,column])+abs(board[5,column])
    row = int(5-colsum)
    if row > -0.5:
        board[row,column] = 1 if color == 'plus' else -1
    return board


def find_legal(board):
    # Identifies columns that are not yet full
    return [i for i in range(7) if abs(board[0,i]) < 0.1]

In [ ]:
def look_for_win(board_,color):
    board_ = board_.copy()
    legal = find_legal(board_)
    winner = -1
    for m in legal:
        bt = update_board(board_.copy(),color,m)
        wi = check_for_win(bt,m)
        if wi[2:] == color:
            winner = m
            break
    return winner

In [ ]:
def find_all_nonlosers(board,color):
    if color == 'plus':
        opp = 'minus'
    else:
        opp = 'plus'
    legal = find_legal(board)
    poss_boards = [update_board(board,color,l) for l in legal]
    poss_legal = [find_legal(b) for b in poss_boards]
    allowed = []
    for i in range(len(legal)):
        wins = [j for j in poss_legal[i] if check_for_win(update_board(poss_boards[i],opp,j),j) != 'nobody']
        if len(wins) == 0:
            allowed.append(legal[i])
    return allowed

In [ ]:
def rollout(board, next_player):
    winner = 'nobody'
    player = next_player
    while winner == 'nobody':
        legal = find_legal(board)
        if len(legal) == 0: return 'tie'
        winning_move = look_for_win(board, player)

        if winning_move != -1:
            move = winning_move
        else:
            # Fallback to random (or your center bias code)
            move = random.choice(legal)
        # ---------------------------------------------

        board = update_board(board, player, move)
        winner = check_for_win(board, move)
        player = 'minus' if player == 'plus' else 'plus'

    return winner

In [ ]:
def back_prop(winner,path,color0,md):
    for i in range(len(path)):
        board_temp = path[i]

        md[board_temp][0]+=1
        if winner[2]==color0[0]:
            if i % 2 == 1:
                md[board_temp][1] += 1
            else:
                md[board_temp][1] -= 1
        elif winner[2]=='e': # tie
            # md[board_temp][1] += 0
            pass
        else:
            if i % 2 == 1:
                md[board_temp][1] -= 1
            else:
                md[board_temp][1] += 1

In [ ]:
def check_for_win(board,col):
    # this code is faster than the above code, but it requires knowing where the last checker was dropped
    # it may seem extreme, but in MCTS this function is called more than anything and actually makes up
    # a large portion of total time spent finding a good move.  So every microsecond is worth saving!
    nrow = 6
    ncol = 7
    # take advantage of knowing what column was last played in...need to check way fewer possibilities
    colsum = abs(board[0,col])+abs(board[1,col])+abs(board[2,col])+abs(board[3,col])+abs(board[4,col])+abs(board[5,col])
    row = int(6-colsum)
    if row+3<6:
        vert = board[row,col] + board[row+1,col] + board[row+2,col] + board[row+3,col]
        if vert == 4:
            return 'v-plus'
        elif vert == -4:
            return 'v-minus'
    if col+3<7:
        hor = board[row,col] + board[row,col+1] + board[row,col+2] + board[row,col+3]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if col-1>=0 and col+2<7:
        hor = board[row,col-1] + board[row,col] + board[row,col+1] + board[row,col+2]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if col-2>=0 and col+1<7:
        hor = board[row,col-2] + board[row,col-1] + board[row,col] + board[row,col+1]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if col-3>=0:
        hor = board[row,col-3] + board[row,col-2] + board[row,col-1] + board[row,col]
        if hor == 4:
            return 'h-plus'
        elif hor == -4:
            return 'h-minus'
    if row < 3 and col < 4:
        DR = board[row,col] + board[row+1,col+1] + board[row+2,col+2] + board[row+3,col+3]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row-1>=0 and col-1>=0 and row+2<6 and col+2<7:
        DR = board[row-1,col-1] + board[row,col] + board[row+1,col+1] + board[row+2,col+2]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row-2>=0 and col-2>=0 and row+1<6 and col+1<7:
        DR = board[row-2,col-2] + board[row-1,col-1] + board[row,col] + board[row+1,col+1]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row-3>=0 and col-3>=0:
        DR = board[row-3,col-3] + board[row-2,col-2] + board[row-1,col-1] + board[row,col]
        if DR == 4:
            return 'd-plus'
        elif DR == -4:
            return 'd-minus'
    if row+3<6 and col-3>=0:
        DL = board[row,col] + board[row+1,col-1] + board[row+2,col-2] + board[row+3,col-3]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    if row-1 >= 0 and col+1 < 7 and row+2<6 and col-2>=0:
        DL = board[row-1,col+1] + board[row,col] + board[row+1,col-1] + board[row+2,col-2]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    if row-2 >=0 and col+2<7 and row+1<6 and col-1>=0:
        DL = board[row-2,col+2] + board[row-1,col+1] + board[row,col] + board[row+1,col-1]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    if row-3>=0 and col+3<7:
        DL = board[row-3,col+3] + board[row-2,col+2] + board[row-1,col+1] + board[row,col]
        if DL == 4:
            return 'd-plus'
        elif DL == -4:
            return 'd-minus'
    return 'nobody'

In [ ]:
def mcts(board_temp,color0,nsteps):
    # nsteps is a parameter that determines the skill (and slowness) of the player
    # bigger values of nsteps means the player is better, but also slower to figure out a move.
    board = board_temp.copy()
    ##############################################
    winColumn = look_for_win(board,color0) # check to find a winning column
    if winColumn > -0.5:
        return winColumn # if there is one - play that!
    legal0 = find_all_nonlosers(board,color0) # find all moves that won't immediately lead to your opponent winning
    if len(legal0) == 0: # if you can't block your opponent - just find the 'best' losing move
        legal0 = find_legal(board)
    ##############################################
    # the code above, in between the hash rows, is not part of traditional MCTS
    # but it makes it better and faster - so I included it!
    # MCTS occasionally makes stupid mistakes
    # like not dropping the checker on a winning column, or not blocking an obvious opponent win
    # this avoids a little bit of that stupidity!
    # we could also add this logic to the rest of the MCTS and rollout functions - I just haven't done that yet...
    # feel free to experiment!
    mcts_dict = {tuple(board.ravel()):[0,0]}
    for ijk in range(nsteps):
        color = color0
        winner = 'nobody'
        board_mcts = board.copy()
        path = [tuple(board_mcts.ravel())]
        while winner == 'nobody':
            legal = find_legal(board_mcts)
            if len(legal) == 0:
                winner = 'tie'
                back_prop(winner,path,color0,mcts_dict)
                break
            board_list = []
            for col in legal:
                board_list.append(tuple(update_board(board_mcts,color,col).ravel()))
            for bl in board_list:
                if bl not in mcts_dict.keys():
                    mcts_dict[bl] = [0,0]
            ucb1 = np.zeros(len(legal))
            for i in range(len(legal)):
                num_denom = mcts_dict[board_list[i]]
                if num_denom[0] == 0:
                    ucb1[i] = 10*nsteps
                else:
                    ucb1[i] = num_denom[1]/num_denom[0] + 2*np.sqrt(np.log(mcts_dict[path[-1]][0])/mcts_dict[board_list[i]][0])
            chosen = np.argmax(ucb1)

            board_mcts = update_board(board_mcts,color,legal[chosen])
            path.append(tuple(board_mcts.ravel()))
            winner = check_for_win(board_mcts,legal[chosen])
            if winner[2]==color[0]:
                back_prop(winner,path,color0,mcts_dict)
                break
            if color == 'plus':
                color = 'minus'
            else:
                color = 'plus'
            if mcts_dict[tuple(board_mcts.ravel())][0] == 0:
                winner = rollout(board_mcts,color)
                back_prop(winner,path,color0,mcts_dict)
                break

    maxval = -np.inf
    best_col = -1
    for col in legal0:
        board_temp = tuple(update_board(board,color0,col).ravel())
        num_denom = mcts_dict[board_temp]
        if num_denom[0] == 0:
            compare = -np.inf
        else:
            compare = num_denom[1] / num_denom[0]
        if compare > maxval:
            maxval = compare
            best_col = col
    return (best_col)


In [ ]:
import numpy as np
import random
import os

# --- 1. PERSPECTIVE-AWARE CNN PREPARATION ---
def prepare_for_cnn(board, current_player):
    """
    Converts 6x7 board into 6x7x2 for neural network training.
    Channel 0: 1s where CURRENT player has pieces.
    Channel 1: 1s where OPPONENT has pieces.
    """
    cnn_input = np.zeros((6, 7, 2), dtype=np.float32)

    if current_player == 'plus':
        cnn_input[:, :, 0] = (board == 1).astype(np.float32)
        cnn_input[:, :, 1] = (board == -1).astype(np.float32)
    else:
        # If it's the 'minus' player's turn, flip the perspective
        cnn_input[:, :, 0] = (board == -1).astype(np.float32)
        cnn_input[:, :, 1] = (board == 1).astype(np.float32)

    return cnn_input

import multiprocessing
from multiprocessing import Pool, cpu_count
import random
import numpy as np
from tqdm import tqdm # Optional: for a nice progress bar

# --- WORKER FUNCTION (Runs one game) ---
def play_single_game(args):
    """
    Runs a single game of Connect 4.
    Args matches the tuple passed from the main loop: (game_seed, mcts_skill)
    """
    game_id, mcts_skill = args

    # CRITICAL: Re-seed random number generators for each process
    # preventing all cores from playing the exact same game.
    random.seed(game_id)
    np.random.seed(game_id)

    # --- ORIGINAL LOGIC STARTS HERE ---
    board = np.zeros((6,7))
    player = 'plus'
    last_move = 0

    game_X = []
    game_Y = []

    # Diversity: Play 2-6 random moves first
    for _ in range(random.randint(2, 6)):
        legal = find_legal(board)
        if not legal: break
        last_move = random.choice(legal)
        board = update_board(board, player, last_move)
        if check_for_win(board, last_move) != 'nobody':
            break
        player = 'minus' if player == 'plus' else 'plus'

    # If random moves didn't end the game, start MCTS
    if check_for_win(board, last_move) == 'nobody':
        while True:
            legal = find_legal(board)
            if not legal: break

            # MCTS finds the 'best' move
            best_move = mcts(board, player, mcts_skill)

            # Record Data
            game_X.append(prepare_for_cnn(board, player))
            game_Y.append(best_move)

            # Make Move
            board = update_board(board, player, best_move)
            if check_for_win(board, best_move) != 'nobody':
                break
            player = 'minus' if player == 'plus' else 'plus'

    return game_X, game_Y

# --- MAIN PARALLEL DRIVER ---
def build_mcts_dataset_parallel(num_games=700, mcts_skill=400):
    # Detect available CPU cores (Colab usually gives 2, Colab Pro gives 4-8)
    num_cores = cpu_count()
    print(f" Starting parallel generation on {num_cores} cores...")

    # Prepare arguments for each game (game_id ensures unique random seeds)
    # We use a large offset for game_id to avoid overlapping with previous runs if you want
    base_seed = random.randint(0, 100000)
    args_list = [(base_seed + i, mcts_skill) for i in range(num_games)]

    X_data = []
    Y_data = []

    # Create a pool of workers
    with Pool(num_cores) as pool:
        # imap_unordered is often faster as it yields results as soon as they finish
        # tqdm adds a progress bar so you know it's working
        results = list(tqdm(pool.imap(play_single_game, args_list), total=num_games))

    # Aggregate results from all games
    print("Aggregation results...")
    for g_x, g_y in results:
        X_data.extend(g_x)
        Y_data.extend(g_y)

    return np.array(X_data), np.array(Y_data)



In [ ]:

# --- EXECUTION ---
if __name__ == '__main__': # Best practice for multiprocessing
    X, Y = build_mcts_dataset_parallel(num_games=5000, mcts_skill=900)

    timestamp = int(time.time())

    # # Save with the timestamp in the name
    np.save(os.path.join(save_path, f'X_train_{timestamp}.npy'), X)
    np.save(os.path.join(save_path, f'Y_train_{timestamp}.npy'), Y)

    print(f"Saved data with ID: {timestamp}")


 Starting parallel generation on 8 cores...


100%|██████████| 5000/5000 [3:59:28<00:00,  2.87s/it]


Aggregation results...
Saved data with ID: 1769202555


In [ ]:
# X_train = np.load('X_train_1768989520.npy')
# Y_train = np.load('Y_train_1768989520.npy')

In [ ]:
# X_train.shape

In [ ]:
# X_train = np.load('X_train.npy')
# Y_train = np.load('Y_train.npy')
# import matplotlib.pyplot as plt
# import numpy as np

# def visualize_test_games(X_data, Y_data):
#     game_count = 1
#     step_count = 1

#     print(f"--- GAME {game_count} START ---")

#     for i in range(len(X_data)):
#         # 1. Reconstruct Board: Channel 0 is Current Player (1), Channel 1 is Opponent (-1)
#         # We assume Current Player = 1 for visualization logic
#         board_state = X_data[i][:, :, 0] - X_data[i][:, :, 1]

#         # 2. Count pieces to detect if a new game started
#         # (If current board has FEWER pieces than the previous step, it's a new game)
#         n_pieces = np.sum(np.abs(board_state))
#         if i > 0:
#             prev_board = X_data[i-1][:, :, 0] - X_data[i-1][:, :, 1]
#             n_prev = np.sum(np.abs(prev_board))
#             if n_pieces < n_prev:
#                 game_count += 1
#                 step_count = 1
#                 print("\n" + "="*40)
#                 print(f"--- GAME {game_count} START ---")
#                 print("="*40)

#         # 3. Predict the Outcome
#         # We apply the move 'Y[i]' to the board and see if it wins
#         temp_board = board_state.copy()

#         # We need to simulate the 'drop' physics
#         col = Y_data[i]
#         # Calculate row (same logic as your update_board function)
#         colsum = np.sum(np.abs(temp_board[:, col]))
#         row = int(5 - colsum)

#         outcome_msg = ""
#         is_win = False

#         if row >= 0:
#             temp_board[row, col] = 1 # 1 is always 'Current Player' in this view

#             # Check for win using your existing function (if available)
#             # OR simple manual check for 4-in-a-row of 1s
#             # (Reusing your notebook's logic ideally, but here's a quick check logic)
#             # We assume 'check_for_win' function exists in your notebook
#             try:
#                 # We need to pass the board with 1s and -1s.
#                 # Your check_for_win returns strings like 'v-plus' or 'nobody'
#                 # In this relative view, 'plus' means Current Player won.
#                 win_res = check_for_win(temp_board, col)
#                 if 'plus' in win_res:
#                     outcome_msg = " -> WINS THE GAME! (Player 1/Red)"
#                     is_win = True
#                 elif 'minus' in win_res:
#                     # Should be impossible for opponent to win on MY turn
#                     outcome_msg = " -> LOST? (Opponent Won)"
#             except:
#                 outcome_msg = " (Could not check win - function missing)"

#         # 4. Print Status
#         print(f"Game {game_count}, Step {step_count}: AI plays Column {col}{outcome_msg}")

#         # Optional: Uncomment these lines to plot every winning board
#         # if is_win:
#         #     plt.figure(figsize=(3,3))
#         #     plt.imshow(temp_board, cmap='RdYlGn', vmin=-1, vmax=1)
#         #     plt.title(f"Game {game_count} Winner")
#         #     plt.axis('off')
#         #     plt.show()

#         step_count += 1

# # Run the visualization
# visualize_test_games(X, Y)

In [ ]:
# import numpy as np

# --- 1. Helper to see the board nicely ---
def print_readable_board(board):
    print("\n  0  1  2  3  4  5  6")
    print("---------------------")
    for row in range(6):
        row_str = "|"
        for col in range(7):
            val = board[row, col]
            if val == 1:
                row_str += " X " # You (Player 1)
            elif val == -1:
                row_str += " O " # AI (Player -1)
            else:
                row_str += " . "
        print(row_str + "|")
    print("---------------------")

# --- 2. The Game Loop ---
def play_game(ai_skill=500):
    board = np.zeros((6, 7))
    game_over = False

    # You are 'plus' (1), AI is 'minus' (-1)
    # You go first!
    turn = 'human'
    player_color = 'plus' # Start with plus

    print(f"GAME START! You are X. AI (Skill {ai_skill}) is O.")

    while not game_over:
        print_readable_board(board)

        legal_moves = find_legal(board)
        if not legal_moves:
            print("It's a TIE!")
            break

        if turn == 'human':
            # --- HUMAN TURN ---
            try:
                col = int(input(f"Your move {legal_moves}: "))
                if col not in legal_moves:
                    print("Invalid move! Try again.")
                    continue
            except ValueError:
                print("Please enter a number.")
                continue

            board = update_board(board, 'plus', col)
            winner = check_for_win(board, col)

            if winner != 'nobody':
                print_readable_board(board)
                print(">>> YOU WIN! <<<")
                game_over = True
            else:
                turn = 'ai'

        else:
            # --- AI TURN ---
            print(f"AI is thinking (Skill={ai_skill})...")
            # Note: mcts takes the board and the CURRENT player's color
            # Since AI is playing 'minus', we pass 'minus'
            ai_move = mcts(board, 'minus', ai_skill)

            print(f"AI chooses column {ai_move}")
            board = update_board(board, 'minus', ai_move)
            winner = check_for_win(board, ai_move)

            if winner != 'nobody':
                print_readable_board(board)
                print(">>> AI WINS! <<<")
                game_over = True
            else:
                turn = 'human'

# --- 3. Start Playing ---
# Adjust skill: 50 is easy, 200 is medium, 800 is hard (but slow)
play_game(ai_skill=150)

GAME START! You are X. AI (Skill 150) is O.

  0  1  2  3  4  5  6
---------------------
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
---------------------
Your move [0, 1, 2, 3, 4, 5, 6]: 3

  0  1  2  3  4  5  6
---------------------
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  X  .  .  . |
---------------------
AI is thinking (Skill=150)...
AI chooses column 1

  0  1  2  3  4  5  6
---------------------
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  O  .  X  .  .  . |
---------------------
Your move [0, 1, 2, 3, 4, 5, 6]: 1

  0  1  2  3  4  5  6
---------------------
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  .  .  .  .  .  . |
| .  X  .  .  .  .  . |
| .  O  .  X  .  .  . |
--------------

In [ ]:
# X_train = np.load(os.path.join(save_path, 'X_train.npy'))
# Y_train = np.load(os.path.join(save_path, 'Y_train.npy'))

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# def visualize_test_games(X_data, Y_data):
#     game_count = 1
#     step_count = 1

#     print(f"--- GAME {game_count} START ---")

#     for i in range(len(X_data)):
#         # 1. Reconstruct Board: Channel 0 is Current Player (1), Channel 1 is Opponent (-1)
#         # We assume Current Player = 1 for visualization logic
#         board_state = X_data[i][:, :, 0] - X_data[i][:, :, 1]

#         # 2. Count pieces to detect if a new game started
#         # (If current board has FEWER pieces than the previous step, it's a new game)
#         n_pieces = np.sum(np.abs(board_state))
#         if i > 0:
#             prev_board = X_data[i-1][:, :, 0] - X_data[i-1][:, :, 1]
#             n_prev = np.sum(np.abs(prev_board))
#             if n_pieces < n_prev:
#                 game_count += 1
#                 step_count = 1
#                 print("\n" + "="*40)
#                 print(f"--- GAME {game_count} START ---")
#                 print("="*40)

#         # 3. Predict the Outcome
#         # We apply the move 'Y[i]' to the board and see if it wins
#         temp_board = board_state.copy()

#         # We need to simulate the 'drop' physics
#         col = Y_data[i]
#         # Calculate row (same logic as your update_board function)
#         colsum = np.sum(np.abs(temp_board[:, col]))
#         row = int(5 - colsum)

#         outcome_msg = ""
#         is_win = False

#         if row >= 0:
#             temp_board[row, col] = 1 # 1 is always 'Current Player' in this view

#             # Check for win using your existing function (if available)
#             # OR simple manual check for 4-in-a-row of 1s
#             # (Reusing your notebook's logic ideally, but here's a quick check logic)
#             # We assume 'check_for_win' function exists in your notebook
#             try:
#                 # We need to pass the board with 1s and -1s.
#                 # Your check_for_win returns strings like 'v-plus' or 'nobody'
#                 # In this relative view, 'plus' means Current Player won.
#                 win_res = check_for_win(temp_board, col)
#                 if 'plus' in win_res:
#                     outcome_msg = " -> WINS THE GAME! (Player 1/Red)"
#                     is_win = True
#                 elif 'minus' in win_res:
#                     # Should be impossible for opponent to win on MY turn
#                     outcome_msg = " -> LOST? (Opponent Won)"
#             except:
#                 outcome_msg = " (Could not check win - function missing)"

#         # 4. Print Status
#         print(f"Game {game_count}, Step {step_count}: AI plays Column {col}{outcome_msg}")

#         # Optional: Uncomment these lines to plot every winning board
#         # if is_win:
#         #     plt.figure(figsize=(3,3))
#         #     plt.imshow(temp_board, cmap='RdYlGn', vmin=-1, vmax=1)
#         #     plt.title(f"Game {game_count} Winner")
#         #     plt.axis('off')
#         #     plt.show()

#         step_count += 1

# # Run the visualization
# visualize_test_games(X, Y)